In [7]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime

In [14]:
class FaceRecognitionSystem:
    def __init__(self, known_faces_dir="known_faces"):
        self.known_faces_dir = known_faces_dir
        self.known_face_encodings = []
        self.known_face_names = []
        self.load_known_faces()
        
    def load_known_faces(self):
        """Load and encode known faces from the directory"""
        for filename in os.listdir(self.known_faces_dir):
            if filename.endswith((".jpg", ".jpeg", ".png")):
                # Load the image
                image_path = os.path.join(self.known_faces_dir, filename)
                image = face_recognition.load_image_file(image_path)
                
                # Get face encoding
                face_encoding = face_recognition.face_encodings(image)[0]
                
                # Store encoding and name (filename without extension)
                self.known_face_encodings.append(face_encoding)
                self.known_face_names.append(os.path.splitext(filename)[0])
                
        print(f"Loaded {len(self.known_face_encodings)} known faces")
    
    def process_frame(self, frame):
        """Process a single frame and return the frame with annotations"""
        # Resize frame for faster face recognition
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        
        # Convert BGR to RGB
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        
        # Find faces in the frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        face_names = []
        for face_encoding in face_encodings:
            # Check if the face matches any known face
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"
            
            if True in matches:
                # Use the first match
                first_match_index = matches.index(True)
                name = self.known_face_names[first_match_index]
            
            face_names.append(name)
        
        # Draw results on the frame
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            # Draw rectangle around face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # Draw label below face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), 
                        cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
            
        return frame
    
    def run_recognition(self):
        """Run face recognition on webcam feed"""
        video_capture = cv2.VideoCapture(0)

        try:
            while True:
                ret, frame = video_capture.read()
                if not ret:
                    print("Failed to grab frame")
                    break
                    
                # Process and display frame
                processed_frame = self.process_frame(frame)
                cv2.imshow('Face Recognition', processed_frame)
                
                # Break loop with 'q'
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print("Exited")
                    break
                    
        finally:
            video_capture.release()
            cv2.destroyAllWindows()
    
    def add_known_face(self, image_path, name):
        """Add a new known face to the system"""
        # Load and encode the face
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]
        
        # Save the image to known_faces directory
        extension = os.path.splitext(image_path)[1]
        new_path = os.path.join(self.known_faces_dir, f"{name}{extension}")
        os.makedirs(self.known_faces_dir, exist_ok=True)
        cv2.imwrite(new_path, cv2.imread(image_path))
        
        # Add to known faces
        self.known_face_encodings.append(face_encoding)
        self.known_face_names.append(name)
        print(f"Added {name} to known faces")


In [17]:
face_system = FaceRecognitionSystem()
face_system.add_known_face("C:\\Users\\IT WORLD\\Desktop\\known_faces\\Sahil.jpg", "Sahil")

Loaded 1 known faces
Added Sahil to known faces


In [18]:
face_system.run_recognition()

Exited
